In [4]:
import re
import time
from itertools import permutations

# Function to preprocess and parse graph data from a file
def preprocess_and_parse(file_path):
    """
    Preprocesses and parses the graph data from a file.

    Args:
    file_path (str): Path to the file containing graph data.

    Returns:
    dict: Graph data represented as an adjacency list.
    """
    graph = {}
    with open(file_path, 'r') as file:
        for line in file:
            # Parse each line of the file
            match = re.match(r'(\d+),\{(.*)\},([\d.-]+)', line)
            if match:
                vertex_id = int(match.group(1))
                parent_set_str = match.group(2)
                cost = float(match.group(3))
                parent_set = tuple(map(int, parent_set_str.split(','))) if parent_set_str else ()

                # Check if vertex exists in graph
                if vertex_id not in graph:
                    graph[vertex_id] = {}

                # Store cost for the corresponding parent set
                graph[vertex_id][parent_set] = cost



    return graph


def reduce_data(graph):
    """
    Reduces the size of the graph by aggregating similar vertices and parent sets.

    Args:
    graph (dict): Graph data represented as an adjacency list.

    Returns:
    dict: Reduced graph data.
    """
    reduced_graph = {}
    for vertex_id, parents in graph.items():
        # Identify similar parent sets and aggregate costs
        reduced_parents = {}
        for parent_set, cost in parents.items():
            if parent_set in reduced_parents:
                reduced_parents[parent_set] = min(reduced_parents[parent_set], cost)  # Choose minimum cost
            else:
                reduced_parents[parent_set] = cost
        reduced_graph[vertex_id] = reduced_parents
    return reduced_graph


def calc_cost(graph, ordering):
    """
    Calculates the total cost for a given ordering.

    Args:
    graph (dict): Graph data represented as an adjacency list.
    ordering (list): The ordering of vertices.

    Returns:
    float: The total cost of the ordering.
    """
    total_cost = 0
    for v in ordering:
        min_cost = float('inf')
        if v in graph:  # Check if vertex exists in the graph
            for parent_set, cost in graph[v].items():
                if set(parent_set).issubset(ordering[:ordering.index(v)]):
                    min_cost = min(min_cost, cost)
        total_cost += min_cost
    return total_cost

# Breadth-First Search (BFS) for finding the optimal ordering
def bfs(graph):
    """
    Performs Breadth-First Search (BFS) to find the optimal ordering.

    Args:
    graph (dict): Graph data represented as an adjacency list.
    """
    start_time = time.time()
    vertices = list(graph.keys())
    min_cost = float('inf')
    optimal_ordering = None
    for ordering in permutations(vertices):
        current_cost = calc_cost(graph, ordering)
        if current_cost < min_cost:
            min_cost = current_cost
            optimal_ordering = ordering
    end_time = time.time()
    print("\nBFS Optimal Ordering:", optimal_ordering)
    print("BFS Total Cost:", min_cost)
    print("Execution Time: {:.6f} seconds".format(end_time - start_time))

# Depth-First Search (DFS) for finding the optimal ordering
def dfs_recursive(graph, curr_ordering, visited, min_cost, optimal_ordering):
    """
    Recursive function for Depth-First Search (DFS) to find the optimal ordering.

    Args:
    graph (dict): Graph data represented as an adjacency list.
    curr_ordering (list): The current ordering of vertices.
    visited (set): Set of visited vertices.
    min_cost (list): List to store the minimum cost found so far.
    optimal_ordering (list): List to store the optimal ordering found so far.
    """
    if len(visited) == len(graph):
        curr_cost = calc_cost(graph, curr_ordering)
        if curr_cost < min_cost[0]:
            min_cost[0] = curr_cost
            optimal_ordering[:] = curr_ordering
        return

    for v in graph.keys():
        if v not in visited:
            visited.add(v)
            dfs_recursive(graph, curr_ordering + [v], visited, min_cost, optimal_ordering)
            visited.remove(v)

def dfs(graph):
    """
    Performs Depth-First Search (DFS) to find the optimal ordering.

    Args:
    graph (dict): Graph data represented as an adjacency list.
    """
    min_cost = [float('inf')]
    optimal_ordering = []
    start_time = time.time()
    dfs_recursive(graph, [], set(), min_cost, optimal_ordering)
    end_time = time.time()
    print("\nDFS Optimal Ordering:", optimal_ordering)
    print("DFS Total Cost:", min_cost[0])
    print("Execution Time: {:.6f} seconds".format(end_time - start_time))

# Uniform Cost Search (UCS) for finding the optimal ordering
def ucs(graph):
    """
    Performs Uniform Cost Search (UCS) to find the optimal ordering.

    Args:
    graph (dict): Graph data represented as an adjacency list.
    """
    start_time = time.time()

    nodes_to_expand = [(0, [], set())]  # (cost, ordering, visited)
    while nodes_to_expand:
        nodes_to_expand.sort(key=lambda x: x[0])  # Sort based on cost
        curr_cost, curr_ordering, visited = nodes_to_expand.pop(0)
        if len(visited) == len(graph):
            print("\nUCS Optimal Ordering:", curr_ordering)
            print("UCS Total Cost:", curr_cost)
            end_time = time.time()
            print("Execution Time: {:.6f} seconds".format(end_time - start_time))
            return
        for v in graph.keys():
            if v not in visited:
                new_ordering = curr_ordering + [v]
                new_visited = visited.copy()
                new_visited.add(v)
                new_cost = calc_cost(graph, new_ordering)
                nodes_to_expand.append((new_cost, new_ordering, new_visited))

file_path = 'data0.txt'
graph_data = preprocess_and_parse(file_path)
reduced_graph_data = reduce_data(graph_data)
bfs(graph_data)
dfs(graph_data)
ucs(graph_data)



BFS Optimal Ordering: (4, 2, 5, 3, 1)
BFS Total Cost: 465.43399999999997
Execution Time: 0.006172 seconds

DFS Optimal Ordering: [4, 2, 5, 3, 1]
DFS Total Cost: 465.43399999999997
Execution Time: 0.006001 seconds

UCS Optimal Ordering: [5, 3, 4, 1, 2]
UCS Total Cost: 465.43399999999997
Execution Time: 0.011996 seconds
